시공간 정보로부터 사고위험도(ECLO)를 예측하는 AI 알고리즘

In [ ]:
!pip install xgboost
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split


import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

train_org = pd.read_csv('/content/drive/MyDrive/colab/daegu_traffic/train.csv')
test_org = pd.read_csv('/content/drive/MyDrive/colab/daegu_traffic/test.csv')
train_org = train_org.drop(columns=train_org.iloc[:,8:18])
train_org.info()
test_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      39609 non-null  object
 1   사고일시    39609 non-null  object
 2   요일      39609 non-null  object
 3   기상상태    39609 non-null  object
 4   시군구     39609 non-null  object
 5   도로형태    39609 non-null  object
 6   노면상태    39609 non-null  object
 7   사고유형    39609 non-null  object
 8   사망자수    39609 non-null  int64 
 9   중상자수    39609 non-null  int64 
 10  경상자수    39609 non-null  int64 
 11  부상자수    39609 non-null  int64 
 12  ECLO    39609 non-null  int64 
dtypes: int64(5), object(8)
memory usage: 3.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   사고일시    10963 non-null  object
 2   요일      10963 non-null  object
 3   기상상태    

In [ ]:
#텍스트 나누기

train_df = train_org.copy()
test_df = test_org.copy()

train_df["사고일시"] = pd.to_datetime(train_org["사고일시"])
train_df['연'] = train_df['사고일시'].dt.year
train_df['월'] = train_df['사고일시'].dt.month
train_df['일'] = train_df['사고일시'].dt.day
train_df['시간'] = train_df['사고일시'].dt.hour

test_df["사고일시"] = pd.to_datetime(train_org["사고일시"])
test_df['연'] = train_df['사고일시'].dt.year
test_df['월'] = train_df['사고일시'].dt.month
test_df['일'] = train_df['사고일시'].dt.day
test_df['시간'] = train_df['사고일시'].dt.hour

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사망자수',\n       '중상자수', '경상자수', '부상자수', 'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월',\n       '일', '시간'],\n      dtype='object')"

"columns of train_df : Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사망자수', '중상자수',\n       '경상자수', '부상자수', 'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간',\n       '도시', '구', '동'],\n      dtype='object')"

"columns of train_df : Index(['ID', '사고일시', '요일', '기상상태', '노면상태', '사고유형', '사망자수', '중상자수', '경상자수',\n       '부상자수', 'ECLO', '연', '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '사고일시', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시',\n       '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

트레인 테스트에 cctv정보추가

In [ ]:

child_df = pd.read_csv('/content/drive/MyDrive/colab/daegu_traffic/external_open/대구_어린이_보호_구역_정보.csv', encoding='cp949')
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_df[['도시', '구', '동','번지']] = child_df['소재지지번주소'].str.extract(location_pattern)
child_df = child_df.drop(columns=['소재지지번주소'])

child_cctv_df = {'동':[],
                 '어린이CCTV설치대수':[]}
child_cctv_df = pd.DataFrame(child_cctv_df)

#동마다 CCTV 갯수 넣을꺼
temp_list = []
for i in child_df['동'].unique() :
  temp_list.append(len(child_df[child_df['동']==i].loc[:,'CCTV설치대수']))

child_cctv_df['동'] = child_df['동'].unique()
child_cctv_df['어린이CCTV설치대수'] = temp_list
child_cctv_df = child_cctv_df.dropna(axis=0) #nan값이 하나 있어서

#train, test 어린이cctv여부 추가

train_df = pd.merge(train_df, child_cctv_df, on='동', how='left')
train_df = train_df.fillna(0)

test_df = pd.merge(test_df, child_cctv_df, on='동', how='left')
test_df = test_df.fillna(0)

cctv_df = pd.read_csv('/content/drive/MyDrive/colab/daegu_traffic/external_open/대구_CCTV_정보.csv', encoding='cp949')
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

cctv_df[['도시', '구', '동','번지']] = cctv_df['소재지지번주소'].str.extract(location_pattern)
cctv_df = cctv_df.drop(columns=['소재지지번주소'])

cctv_list = {'동':[],
             'CCTV개수':[]}
cctv_list = pd.DataFrame(cctv_list)
#동마다 CCTV 갯수 넣을꺼
temp_list = []
for i in cctv_df['동'].unique() :
  temp_list.append(len(cctv_df[(cctv_df['동']==i)&((cctv_df['단속구분']==2)|(cctv_df['단속구분']==1)|(cctv_df['단속구분']==99))]))

cctv_list['동'] = cctv_df['동'].unique()
cctv_list['CCTV개수'] = temp_list
cctv_list = cctv_list.dropna(axis=0) #nan값이 하나 있어서

#train, test cctv_list 추가

train_df = pd.merge(train_df, cctv_list, on='동', how='left')
train_df = train_df.fillna(0)

test_df = pd.merge(test_df, cctv_list, on='동', how='left')
test_df = test_df.fillna(0)

#동마다 CCTV 제한속도 평균
speed_df = {'동':[],
             '평균속도':[]}
speed_df = pd.DataFrame(speed_df)
speed_list = []
for i in cctv_df['동'].unique() :
  temp_list = []
  count_list = []
  temp = 0
  sum_list = []
  for j in [30,50,60,70,80,100] :
    temp_list.append(len(cctv_df[(cctv_df['제한속도']==j)&(cctv_df['동']== i)])*j)
    temp += (len(cctv_df[(cctv_df['제한속도']==j)&(cctv_df['동']== i)]))
    #print(temp_list)
    #print(temp)

  sum_list = sum(temp_list)
  if temp != 0 :
    speed_list.append(sum_list/temp)
  else:
    speed_list.append(38)
  #print(speed_list)
#len(speed_list)

speed_df['동'] = cctv_df['동'].unique()
speed_df['평균속도'] = speed_list
speed_df = speed_df.dropna(axis=0) #nan값이 하나 있어서

#train, test cctv_list 추가

train_df = pd.merge(train_df, speed_df, on='동', how='left')
print(train_df.shape)
train_df = train_df.fillna(0)
print(train_df.isnull().sum())

test_df = pd.merge(test_df, speed_df, on='동', how='left')
print(test_df.shape)
test_df = test_df.fillna(0)
print(test_df.isnull().sum())


(39609, 23)
ID             0
사고일시           0
요일             0
기상상태           0
노면상태           0
사고유형           0
사망자수           0
중상자수           0
경상자수           0
부상자수           0
ECLO           0
연              0
월              0
일              0
시간             0
도시             0
구              0
동              0
도로형태1          0
도로형태2          0
어린이CCTV설치대수    0
CCTV개수         0
평균속도           0
dtype: int64
(10963, 18)
ID             0
사고일시           0
요일             0
기상상태           0
노면상태           0
사고유형           0
연              0
월              0
일              0
시간             0
도시             0
구              0
동              0
도로형태1          0
도로형태2          0
어린이CCTV설치대수    0
CCTV개수         0
평균속도           0
dtype: int64


기상상태 비이고 노면상태 기타 경우 젖음/습기로 변경<br>
미분류 도로 가장 많은 도로형태로 수정

In [ ]:
#train_df.loc[4185,'노면상태'] = '젖음/습기'
#train_df.loc[5135,'노면상태'] = '젖음/습기'
#train_df.loc[6892,'노면상태'] = '젖음/습기'
#train_df.loc[33750,'노면상태'] = '젖음/습기'
#train_df.loc[35213,'노면상태'] = '젖음/습기'
#train_df.loc[36407,'노면상태'] = '젖음/습기'

train_df.loc[(train_df['기상상태'] == '비') & (train_df['노면상태'] == '기타'), '노면상태'] = '젖음/습기'


train_df.loc[983,'도로형태1'] = '단일로'
train_df.loc[1084,'도로형태1'] = '단일로'
train_df[(train_df['동']=='율하동')]['도로형태1'].value_counts() #율하동은 교차로 사고가 많다.
train_df.loc[12779,'도로형태1'] = '교차로'
train_df.loc[15137,'도로형태1'] = '단일로'
train_df.loc[15742,'도로형태1'] = '단일로'
train_df.loc[16924,'도로형태1'] = '단일로'
train_df.loc[21393,'도로형태1'] = '단일로'
train_df[(train_df['동']=='다사읍')]['도로형태1'].value_counts() #다사읍은 교차로 사고가 많다.
train_df.loc[26090,'도로형태1'] = '교차로'

train_df.loc[(train_df['도로형태2'] == '미분류'), '도로형태2'] = '기타'


In [ ]:
import matplotlib.pyplot as plt
# 여러 조건에 따라 값을 설정하는 함수 정의

def assign_category(value):
    if value in [1,2,3,4]:
        return '새벽'
    elif value in [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,6,5]:
        return '주간'
    else:
        return 'Unknown'

def weekday_category(value):
    if value in ['화요일','수요일','목요일','금요일','월요일']:
        return '주간'
    elif value in ['토요일','일요일']:
        return '주말'
    else:
        return 'Unknown'

# 'Value' 열에 대해 새로운 열 생성
train_df['통합시간'] = train_df['시간'].apply(assign_category)
train_df['통합요일'] = train_df['요일'].apply(weekday_category)
test_df['통합시간'] = test_df['시간'].apply(assign_category)
test_df['통합요일'] = test_df['요일'].apply(weekday_category)

train_df = pd.get_dummies(train_df, columns=['통합시간'],dtype=float)
train_df = pd.get_dummies(train_df, columns=['통합요일'],dtype=float)
test_df = pd.get_dummies(test_df, columns=['통합시간'],dtype=float)
test_df = pd.get_dummies(test_df, columns=['통합요일'],dtype=float)

피처, 타겟 분리 라벨인코딩

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X_num = train_df[['어린이CCTV설치대수','CCTV개수','평균속도']]
scaler = MinMaxScaler()
transform_X_num = scaler.fit_transform(X_num)
transform_X_num = pd.DataFrame(transform_X_num)
transform_test_num = scaler.transform(test_df[['어린이CCTV설치대수','CCTV개수','평균속도']])
transform_test_num = pd.DataFrame(transform_test_num)
X_scaled = pd.concat([train_df, transform_X_num], axis=1)
test_scaled = pd.concat([test_df, transform_test_num], axis=1)
X_scaled.drop(columns=['어린이CCTV설치대수','CCTV개수','평균속도'], inplace=True)
test_scaled.drop(columns=['어린이CCTV설치대수','CCTV개수','평균속도'], inplace=True)

from sklearn.preprocessing import LabelEncoder

#FEATURE = ['기상상태','노면상태','사고유형','구','동','도로형태1','도로형태2','요일','도시']
FEATURE = ['기상상태','노면상태','동','도로형태1','도로형태2','사고유형','요일','구']
for i in FEATURE:
    le = LabelEncoder()
    X_scaled[i]= le.fit_transform(X_scaled[i])
    test_scaled[i]= le.transform(test_scaled[i])

test_scaled

,ID,사고일시,요일,기상상태,노면상태,사고유형,연,월,일,시간,...,ECLO_make,경상자수_make,중상자수_make,통합시간_새벽,통합시간_주간,통합요일_주간,통합요일_주말,0,1,2
0,ACCIDENT_39609,2019-01-01 00:00:00,4.920811,4.712888,4.712481,3.817650,2019,1,1,0,...,5.50000,1.000000,1,0.0,1.0,0.0,1.0,0.083333,0.076923,0.525000
1,ACCIDENT_41450,2019-02-25 17:00:00,4.600906,4.712888,4.712481,3.817650,2019,2,25,17,...,5.50000,1.000000,1,0.0,1.0,1.0,0.0,0.083333,0.076923,0.525000
2,ACCIDENT_46858,2019-07-08 19:00:00,5.210258,4.712888,4.712481,3.817650,2019,7,8,19,...,5.50000,1.000000,1,0.0,1.0,0.0,1.0,0.083333,0.076923,0.525000
3,ACCIDENT_39610,2019-01-01 00:00:00,4.920811,4.712888,4.712481,3.817650,2019,1,1,0,...,4.04918,0.540984,2,0.0,1.0,0.0,1.0,0.166667,0.134615,0.471429
4,ACCIDENT_39741,2019-01-05 12:00:00,4.560688,4.712888,4.712481,3.817650,2019,1,5,12,...,4.04918,0.540984,2,0.0,1.0,1.0,0.0,0.166667,0.134615,0.471429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10512,ACCIDENT_50483,2019-10-08 14:00:00,4.627408,4.712888,4.712481,4.944597,2019,10,8,14,...,6.00000,2.000000,0,0.0,1.0,1.0,0.0,0.000000,0.000000,0.000000
10513,ACCIDENT_50543,2019-10-09 19:00:00,4.600906,4.712888,4.712481,4.006054,2019,10,9,19,...,5.00000,0.000000,1,0.0,1.0,1.0,0.0,0.216667,0.173077,0.433333
10514,ACCIDENT_50555,2019-10-09 23:00:00,4.920811,4.712888,4.712481,4.944597,2019,10,9,23,...,3.50000,0.750000,1,0.0,1.0,0.0,1.0,0.000000,0.019231,0.600000
10515,ACCIDENT_50560,2019-10-10 08:00:00,4.920811,4.712888,4.712481,4.944597,2019,10,10,8,...,4.50000,1.000000,1,0.0,1.0,0.0,1.0,0.000000,0.000000,0.000000


In [ ]:
#X = train_df.drop(columns=['ECLO','부상자수','경상자수','중상자수','사망자수','피해운전자 상해정도','피해운전자 연령','피해운전자 차종','가해운전자 상해정도',
                           #'가해운전자 연령','가해운전자 성별','가해운전자 차종','법규위반','사고유형 - 세부분류','피해운전자 성별','도시'])
X = X_scaled.drop(columns=['ID','도시','ECLO','사망자수','중상자수','경상자수','부상자수'])
y = X_scaled['ECLO']
test_scaled = test_scaled.drop(columns=['ID','도시'])
X

,사고일시,요일,기상상태,노면상태,사고유형,연,월,일,시간,구,...,ECLO_make,경상자수_make,중상자수_make,통합시간_새벽,통합시간_주간,통합요일_주간,통합요일_주말,0,1,2
0,2019-01-01 00:00:00,4.627926,4.712888,4.712481,3.817650,2019,1,1,0,4.541610,...,3.692308,0.5,1,0.0,1.0,1.0,0.0,0.033333,0.019231,0.380000
1,2019-01-19 09:00:00,4.920811,4.712888,4.712481,3.817650,2019,1,19,9,4.541610,...,3.692308,0.5,1,0.0,1.0,0.0,1.0,0.033333,0.019231,0.380000
2,2019-03-08 16:00:00,4.600906,4.712888,4.712481,3.817650,2019,3,8,16,4.541610,...,3.692308,0.5,1,0.0,1.0,1.0,0.0,0.033333,0.019231,0.380000
3,2019-03-27 09:00:00,4.627408,4.712888,4.712481,3.817650,2019,3,27,9,4.541610,...,3.692308,0.5,1,0.0,1.0,1.0,0.0,0.033333,0.019231,0.380000
4,2019-04-04 15:00:00,4.560688,4.712888,4.712481,3.817650,2019,4,4,15,4.541610,...,3.692308,0.5,1,0.0,1.0,1.0,0.0,0.033333,0.019231,0.380000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,2021-12-27 08:00:00,4.698049,4.712888,4.712481,3.817650,2021,12,27,8,4.889534,...,3.000000,1.0,0,0.0,1.0,1.0,0.0,0.000000,0.038462,0.600000
39605,2021-12-27 19:00:00,4.698049,4.712888,4.712481,4.944597,2021,12,27,19,4.727300,...,3.000000,1.0,0,0.0,1.0,1.0,0.0,0.316667,0.230769,0.481818
39606,2021-12-27 20:00:00,4.698049,4.712888,4.712481,3.817650,2021,12,27,20,4.687669,...,3.000000,1.0,0,0.0,1.0,1.0,0.0,0.066667,0.019231,0.300000
39607,2021-12-28 22:00:00,4.627926,4.712888,4.712481,3.817650,2021,12,28,22,4.889534,...,5.000000,0.0,1,0.0,1.0,1.0,0.0,0.000000,0.000000,0.000000


In [ ]:
result = X_scaled.groupby(['동'])['ECLO'].mean()
filtered_result = result[result > 6.8]

In [ ]:
eclo_58 = X_scaled[X_scaled['동'].isin(filtered_result.index)]
eclo_58_test = test_scaled[test_scaled['동'].isin(filtered_result.index)]
eclo_58_test = eclo_58_test.drop(columns=['ID','도시'])
X_58 = eclo_58.drop(columns=['ID','도시','ECLO','사망자수','중상자수','경상자수','부상자수'])
y_58 = eclo_58['ECLO']

In [ ]:
under58_eclo = X_scaled[~(X_scaled['동'].isin(filtered_result.index) | X_scaled['동'].isin(filtered_result.index))]
under58_eclo_test = test_scaled[~(test_scaled['동'].isin(filtered_result.index) | test_scaled['동'].isin(filtered_result.index))]
under58_eclo_test = under58_eclo_test.drop(columns=['ID','도시'])
under_58 = under58_eclo.drop(columns=['ID','도시','ECLO','사망자수','중상자수','경상자수','부상자수'])
undery_58 = under58_eclo['ECLO']

모델 훈련 및 예측

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error

# Define the number of folds for cross-validation
n_splits = 10  # You can adjust this value

# Create an instance of KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

def objective(trial, X, y):
    rmsle_values = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = under_58.iloc[train_idx], under_58.iloc[val_idx]
        y_train, y_val = undery_58.iloc[train_idx], undery_58.iloc[val_idx]

        # XGBoost의 하이퍼파라미터를 설정합니다.
        param = {
            'objective': 'reg:squaredlogerror',
            'eval_metric': 'rmsle',
            'lambda': trial.suggest_float('lambda', 1e-3, 0.1),
            'alpha': trial.suggest_float('alpha', 1e-3, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
            'subsample': trial.suggest_float('subsample', 0.4, 1),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 2, 50),
        }

        # XGBoost 모델을 생성합니다.
        model = xgb.XGBRegressor(**param)

        # 모델을 훈련합니다.
        model.fit(X_train, y_train, eval_set=[(X_train, y_train)], early_stopping_rounds=100,verbose=False)

        # 검증 데이터에 대한 예측을 수행합니다.
        y_pred = model.predict(X_val)

        # RMSE를 계산하여 리스트에 추가합니다.
        rmsle = np.sqrt(mean_squared_log_error(y_val, y_pred))
        rmsle_values.append(rmsle)

    # 평균 RMSE를 반환합니다.
    return np.mean(rmsle_values)

# Optuna를 사용하여 하이퍼파라미터 튜닝을 수행합니다.
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, under_58, undery_58), n_trials=100)

# 최적의 하이퍼파라미터를 출력합니다.
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_params = trial.params

# 최적의 하이퍼파라미터로 최종 모델을 훈련합니다.
final_model = xgb.XGBRegressor(
    objective='reg:squaredlogerror',
    eval_metric='rmsle',
    verbosity=0,
    n_jobs=-1,
    tree_method='hist',
    **best_params
)

final_model.fit(under_58, undery_58, verbose=False)
y_pred = final_model.predict(under58_eclo_test)

In [ ]:
params = {
    'lambda': 0.011825773074770396,
    'alpha': 0.5044643996727611,
    'colsample_bytree': 0.8531532574509729,
    'subsample': 0.8021843456119657,
    'learning_rate': 0.012079406386966347,
    'n_estimators': 772,
    'max_depth': 5,
    'min_child_weight': 2,
    'objective': 'reg:squaredlogerror',
    'eval_metric': 'rmsle',
}

model = xgb.XGBRegressor(**params)

# 모델 훈련
model.fit(under_58, undery_58)

# 테스트 데이터로 예측
y_pred_under = model.predict(under58_eclo_test)



In [ ]:
params = {
    'lambda': 0.011825773074770396,
    'alpha': 0.5044643996727611,
    'colsample_bytree': 0.8531532574509729,
    'subsample': 0.8021843456119657,
    'learning_rate': 0.012079406386966347,
    'n_estimators': 772,
    'max_depth': 5,
    'min_child_weight': 2,
    'objective': 'reg:squaredlogerror',
    'eval_metric': 'rmsle',
}

model = xgb.XGBRegressor(**params)

# 모델 훈련
model.fit(X_58, y_58)

# 테스트 데이터로 예측
y_pred = model.predict(eclo_58_test)



In [ ]:
eclo_58_test['ECLO'] = y_pred
under58_eclo_test['ECLO'] = y_pred_under
pred_sub = pd.concat([eclo_58_test, under58_eclo_test])
pred_sub
pred_sub = pred_sub.sort_index()

In [ ]:
pred_sub = pred_sub['ECLO'].values
pred_sub

array([4.0486975, 3.6323957, 5.0447416, ..., 4.3348036, 4.25082  ,
       4.3396044], dtype=float32)

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/colab/daegu_traffic/sample_submission.csv')
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = pred_sub
baseline_submission

,ID,ECLO
0,ACCIDENT_39609,4.048697
1,ACCIDENT_39610,3.632396
2,ACCIDENT_39611,5.044742
3,ACCIDENT_39612,4.454095
4,ACCIDENT_39613,4.653054
...,...,...
10958,ACCIDENT_50567,4.177457
10959,ACCIDENT_50568,4.305200
10960,ACCIDENT_50569,4.334804
10961,ACCIDENT_50570,4.250820


In [ ]:
baseline_submission.to_csv('xgboost61.csv', index=False)